In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from BaselineRemoval import BaselineRemoval
import tkinter
from sklearn.preprocessing import StandardScaler
import tkinter.messagebox
import customtkinter
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, 
NavigationToolbar2Tk)
import os
import seaborn
from PIL import Image
import random
from tkinter import filedialog
import csv
from scipy.signal import find_peaks
import webbrowser
import joblib
from tkinter import messagebox

In [2]:

class StatisticalAnalysisFrame(customtkinter.CTkFrame):
    def __init__(self, container):
        super().__init__(container)
        
        self.grid_rowconfigure(0, weight=5)
        self.grid_rowconfigure(1, weight=1)
        self.grid_rowconfigure(2, weight=1)
        self.grid_columnconfigure(0, weight=1)

        # Statistical analysis figure
        self.fig = Figure(figsize=(5, 5))
        #self.axis = self.fig.add_subplot(111)
        self.subplot1 = self.fig.add_subplot(121)  # First subplot
        self.subplot2 = self.fig.add_subplot(122)  # Second subplot

        self.canvas = FigureCanvasTkAgg(self.fig, master=self)
        self.canvas.draw()
        self.canvas.get_tk_widget().grid(row=0, column=0, padx=5, pady=5, sticky="nsew")

        # Buttons
        self.buttonFrame = tkinter.Frame(self)
        self.buttonFrame.grid(row=1, column=0, padx=20, pady=5)
        
        self.inputButton = tkinter.Button(self.buttonFrame, text="Input data", command=self.inputData)
        self.inputButton.grid(row=0, column=0, padx=10, pady=40)

        self.analyseButton = tkinter.Button(self.buttonFrame, text="Analyse", command=self.analyse)
        self.analyseButton.grid(row=0, column=1, padx=10, pady=40)
         
        self.gButton = tkinter.Button(self.buttonFrame, text="G-factor", command=self.toggle_plots)
        self.gButton.grid(row=0, column=2, padx=10, pady=40)

        self.saveButton = tkinter.Button(self.buttonFrame, text="Save figure", command=self.saveFigure)
        self.saveButton.grid(row=0, column=3, padx=10, pady=40)
        self.lower_bound = None
        self.upper_bound = None
        
        #self.subplot2_visible = True
    def get_range(self):
        dialog = tkinter.Toplevel(self.master)
        dialog.title("Enter Range")

        tkinter.Label(dialog, text="Enter lower bound of magnetic field range (mT):").grid(row=0, column=0, padx=5, pady=5)
        tkinter.Label(dialog, text="Enter upper bound of magnetic field range (mT):").grid(row=1, column=0, padx=5, pady=5)

        lower_entry = tkinter.Entry(dialog)
        upper_entry = tkinter.Entry(dialog)

        lower_entry.grid(row=0, column=1, padx=5, pady=5)
        upper_entry.grid(row=1, column=1, padx=5, pady=5)

        def on_ok():
            self.lower_bound = float(lower_entry.get())
            self.upper_bound = float(upper_entry.get())
            dialog.destroy()
            #self.analyse()

        ok_button = tkinter.Button(dialog, text="OK", command=on_ok)
        ok_button.grid(row=2, column=0, columnspan=2, pady=10)

        dialog.wait_window(dialog)  # Wait for the dialog to be closed

    def inputData(self):
        files = filedialog.askopenfilenames(parent=self, title='Choose files', filetypes=[("text files", ".txt"), ("csv files", ".csv")])
        self.field = []
        self.intensity = []
        for ele in files:
            if ele.endswith(".csv"):  # Check if the file is a CSV file
                self.data = pd.read_csv(ele)
            elif ele.endswith(".txt"):  # Check if the file is a text file
                values = np.genfromtxt(ele, names=True, skip_header=2)
                self.data = pd.DataFrame(values)
        # Fill missing values with column mean
        for column in self.data.columns:
            col_mean = self.data[column].mean()
            self.data[column] = self.data[column].fillna(col_mean)
        # Drop the fourth column
        if len(self.data.columns) > 3:
            self.data = self.data.drop(self.data.columns[3], axis=1)
            # Rename columns
            self.data.columns = ['index', 'Field', 'Intensity']
        else:
            self.data.columns = ['Field', 'Intensity']

    def analyse(self):
        self.get_range()  # Call get_range() to prompt the user for range
        series = self.data.isnull().sum()
        for index, value in series.items():
            if value != 0:
                col_mean = self.data[index].mean()
                self.data[index] = self.data[index].fillna(col_mean)
       
        self.filtered_data = self.data[(self.data['Field'] >= self.lower_bound) & (self.data['Field'] <= self.upper_bound)]
        #self.filtered_data = self.data[(self.data['Field'] >= 2000) & (self.data['Field'] <= 4000)]
        #print(self.filtered_data)
        self.fig = Figure(figsize=(14, 5))
        # Calculate the g-factor for the filtered data
        self.filtered_data['g-factor'] = 689.470305 / self.filtered_data['Field']
        self.subplot1.clear()
        self.plot1, = self.subplot1.plot(self.filtered_data['g-factor'], self.filtered_data['Intensity'])
        self.subplot1.set_xlabel('g-factor')
        self.subplot1.set_ylabel('Intensity')
        self.subplot1.set_title('Intensity v/s g-factor')
        
       
        self.subplot2.clear()
        self.plot2, = self.subplot2.plot(self.filtered_data['Field'], self.filtered_data['Intensity'])
        self.subplot2.set_xlabel('Magnetic Field (mT)')
        self.subplot2.set_ylabel('Intensity')
        self.subplot2.set_title('Intensity v/s Magnetic Field')

        #self.canvas = FigureCanvasTkAgg(self.fig, master=self)
        self.canvas.draw()
    def toggle_plots(self):
        # Plot the data in the second subplot
        #self.subplot2.plot(self.data['Field'], self.data['Intensity'])
        
        # Plot a line at intensity = 0 in the first subplot (Intensity vs. g-factor)
        zero_intensity_line_g_factor = [0] * len(self.filtered_data['g-factor'])
        self.subplot1.plot(self.filtered_data['g-factor'], zero_intensity_line_g_factor, linestyle='--', color='red', zorder=10)
        crossing_points = []
        intensity_sign = np.sign(self.filtered_data['Intensity'])  # Get the sign of intensity
        print(intensity_sign)
        start_index = self.filtered_data.index[0]
        end_index = self.filtered_data.index[-1]

        print("Start index of intensity_sign:", start_index)
        print("End index of intensity_sign:", end_index)
        #print(start)
        for i in range(start_index+1,end_index):
            if intensity_sign[i - 1] != intensity_sign[i]:
                crossing_points.append(i)
        print('Cross:',crossing_points)
    
        if crossing_points:
            # Take the midpoint of the first crossing point
            crossing_point_index = crossing_points[-1]
            print(crossing_point_index)
            crossing_point = self.filtered_data.loc[crossing_point_index,'g-factor']
            print(crossing_point)
        
            # Plot a vertical line at the crossing point
            self.subplot1.axvline(x=crossing_point, color='green', linestyle='--')
        
            # Calculate the final g-factor
            final_g_factor = crossing_point
        
            #self.subplot1.text(crossing_point, 0, f'Final g-factor: {final_g_factor:.2f}', color='blue')
            self.subplot1.text(0.95, 0.95, f'Final g-factor: {final_g_factor:.2f}', color='blue', ha='right', va='top', transform=self.subplot1.transAxes)

            # Display the g-factor value in a message box
            messagebox.showinfo(title='G-Factor', message=f'The final g-factor is: {final_g_factor:.2f}')

            #self.subplot2_visible = True

        # Redraw both subplots and the canvas
        self.subplot1.figure.canvas.draw()
        self.subplot2.figure.canvas.draw()
        self.canvas.draw()
     
    def saveFigure(self):
        file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png")])
        if file_path:
            self.fig.savefig(file_path)
            messagebox.showinfo(title='Save', message='File saved successfully')
   

In [ ]:
customtkinter.set_appearance_mode("System")  # Modes: "System" (standard), "Dark", "Light"
customtkinter.set_default_color_theme("blue")  # Themes: "blue" (standard), "green", "dark-blue"

class MainFrame(customtkinter.CTkFrame):
    def __init__(self,container):
        
        super().__init__(container)
        
        self.grid_rowconfigure((0 , 1 , 2 , 4) , weight = 0)
        self.grid_rowconfigure(3 , weight = 1)
        self.grid_columnconfigure(0 , weight = 25)
        self.grid_columnconfigure(1 , weight = 0)
        
        # create radiobutton frame
        self.topRightFrame = customtkinter.CTkFrame(self)
        self.topRightFrame.grid(row=0, column=1, rowspan = 2 ,padx=(20, 20), pady=(20, 0), sticky="nsew")
        
        self.inputDataButton = customtkinter.CTkButton(self.topRightFrame , text = "Input Data" , command = self.input_data)
        self.inputDataButton.pack(padx = 20 , pady = 10)
        
        self.preprocessingButton = customtkinter.CTkButton(self.topRightFrame , text = "Data preprocessing" , command = self.preprocessing)
        self.preprocessingButton.pack(padx = 20 , pady = 10)
        
        self.plotButton = customtkinter.CTkButton(self.topRightFrame , text = "Plot" , command = self.plot)
        self.plotButton.pack(padx = 20 , pady = 10)
        
        self.fitGuassianButton = customtkinter.CTkButton(self.topRightFrame , text = " Free Radical Annotation" , command = self.FreeRadical)
        self.fitGuassianButton.pack(padx = 20 , pady = 10)
        
        self.saveCurveButton = customtkinter.CTkButton(self.topRightFrame , text = "Save curve" , command = self.saveCurve)
        self.saveCurveButton.pack(padx = 20 , pady = 10)
        
        # create checkbox and switch frame
        self.bottomRight = customtkinter.CTkFrame(self)
        self.bottomRight.grid(row=2 ,column=1, rowspan = 3 , padx=(20, 20), pady=(20, 20) , sticky = "nsew")
        
        # Image frame
        
        self.imageFrame = customtkinter.CTkFrame(self)
        self.imageFrame.grid(row=0, column=0, rowspan = 4, padx=(20, 0), pady=(20, 20), sticky="nsew")
        
        
        self.fig = Figure(figsize = (5, 5))
        self.axis = self.fig.add_subplot(111)

        self.canvas = FigureCanvasTkAgg(self.fig,master = self.imageFrame)  
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(expand = True , fill = tkinter.BOTH)
        
        # Sliders
        
        self.slider_frame = customtkinter.CTkFrame(self)
        self.slider_frame.grid(row = 4 , column = 0, padx = (20,0) , pady = (0 , 20) , sticky = 'sew')
        
        self.slider_1 = customtkinter.CTkSlider(self.slider_frame, from_= 0, to=1)
        self.slider_1.pack(fill = tkinter.BOTH , expand = True , padx = 20 , pady = 10)
        
        self.slider_2 = customtkinter.CTkSlider(self.slider_frame, from_=0, to=1)
        self.slider_2.pack(fill = tkinter.BOTH, expand = True, padx = 20 , pady = 10)
        
        self.setButton = customtkinter.CTkButton(self.slider_frame , text = 'Set' , command = lambda : self.setXAxis(self.all_smoothed_spectrum , self.slider_1.get() , self.slider_2.get()))
        self.setButton.pack(pady = (0 , 10))
        
        
        # Intitial configuration
        self.plotButton.configure(state = "disabled")
        self.setButton.configure(state="disabled")
        
    def input_data(self):
        files = tkinter.filedialog.askopenfilenames(parent=self, title='Choose files', filetypes=[("text files", ".txt"), ("csv files", ".csv")])
        self.field = []
        self.intensity = []
        for ele in files:
            if ele.endswith(".csv"):  # Check if the file is a CSV file
                self.data = pd.read_csv(ele)
                self.plotButton.configure(state="enabled")  # Enable plot feature
            elif ele.endswith(".txt"):  # Check if the file is a text file
                values = np.genfromtxt(ele, names=True, skip_header=2)
                self.data = pd.DataFrame(values)

        
    def preprocessing(self):
        # Fill missing values with column mean
        for column in self.data.columns:
            col_mean = self.data[column].mean()
            self.data[column] = self.data[column].fillna(col_mean)
        # Drop the fourth column
        if len(self.data.columns) > 3:
            self.data = self.data.drop(self.data.columns[3], axis=1)
            # Rename columns
            self.data.columns = ['index', 'Field [G]', 'Intensity']
        else:
            self.data.columns = ['Field [G]', 'Intensity']
        # Save preprocessed data to a CSV file
        file_path = tkinter.filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
        if file_path:
            self.data.to_csv(file_path, index=False)  
        # Save DataFrame to CSV with specific columns and without index
            tkinter.messagebox.showinfo(title='Save', message='CSV file saved successfully')

        # Enable buttons after saving
        self.setButton.configure(state="enable")
        self.plotButton.configure(state="enable")
                
    def plot(self):
        #self.canvas = FigureCanvasTkAgg(self.fig,master = self)  
        #self.canvas.draw()
        #self.canvas.get_tk_widget().grid(row = 0 , column = 0 , padx = (5 , 5) , pady = (5 , 5) , sticky="nsew")
        self.axis.clear()
        if len(self.data.columns) == 3:
            self.axis.plot(self.data.iloc[:, 1], self.data.iloc[:, 2])
        else:
            self.axis.plot(self.data.iloc[:, 0], self.data.iloc[:, 1])
        self.axis.set_xlabel('Magnetic Field(mT)')
        self.axis.set_ylabel('Intensity')
        self.axis.set_title('Intensity v/s Magnetic Field')
        self.canvas.draw()
        #self.axis = self.fig.add_subplot(111)
        #self.axis.scatter(self.data['Field'] , self.data['Intensity'])
        #plt.plot(self.data['Field'],self.data['Intensity'])
        #plt.show()
    """
    Find distances and heights of peaks in a spectrum.

    Parameters:
    - spectrum: List of intensity values representing the spectrum.
    - threshold: Threshold parameter for peak detection (default is 0.5).

    Returns:
    - distances: List of distances between adjacent peaks.
    - heights: List of heights of the peaks.
    """
    def FreeRadical(self, threshold=0.1):
        spectrum = self.data['Intensity']
        # Find peaks in the spectrum
        peaks, _ = find_peaks(spectrum, height=threshold)
        # Calculate distances between adjacent peaks
        distances = np.diff(peaks)
        # Get heights of the peaks
        height = spectrum[peaks]
        heights = height.values
        print(heights)
        radical_type=[]
        message=""
        if len(heights) == 1:
            # If there is a single peak, the radical is O2-
            midpoint_B = peaks[0] + (peaks[-1] - peaks[0]) / 2
            g_factor = heights[0] / (9.274 * (10 ** (-24)) * midpoint_B)
            radical_type.append("O2-")
            message += "G-factor: N/A\nFree radical type: O2-\n"
        elif len(heights) == 3:
            selected_heights = np.array([heights[0]] * 3)
            if np.allclose(selected_heights, [heights[0]] * 3):
                # Heights of three consecutive peaks are the same
                aN = distances[0]  # Distance between consecutive peaks
                aH = distances[1]  # Distance between peaks due to splitting
                if aH == 0:
                    aH = aN  # If no splitting, aH is equal to aN
                aN_aH = aN / aH  # Ratio of aN to aH
                midpoint_B = peaks[0] + (peaks[-1] - peaks[0]) / 2
                g_factor = heights[0] / (9.274 * (10 ** (-24)) * midpoint_B)
                message+=f"G-factor: {g_factor}\nFree radical type: {radical_type}\n"
        elif len(heights) >= 4:
            selected_heights = np.array([heights[0]] * 4)
            if np.allclose(selected_heights, [heights[0]] * 4):
                # Heights of four consecutive peaks are the same
                aN = distances[0]  # Distance between consecutive peaks
                aH = distances[1]  # Distance between peaks due to splitting
                if aH == 0:
                    aH = aN  # If no splitting, aH is equal to aN
                aN_aH = aN / aH  # Ratio of aN to aH
            
                # Load the machine learning model
                model = joblib.load("model.pkl")
                # Predict radical type using the model
                radical_type_pred = model.predict([[aN, aH, aN_aH]])
                if radical_type_pred == 0:
                    radical_type.append("O2H")
                midpoint_B = peaks[0] + (peaks[-1] - peaks[0]) / 2
                g_factor = heights[0] / (9.274 * (10 ** (-24)) * midpoint_B)
                message+=f"G-factor: {g_factor}\nFree radical type: {radical_type}\n"
            if np.allclose(selected_heights, [heights[0], heights[0]*2, heights[0]*2, heights[0]]):
                # Heights of four consecutive large peaks are in the ratio 1:3:3:1
                aN = distances[0]  # Distance between larger peaks
                aH = distances[1]  # Distance between peaks due to splitting
                if aH == 0:
                    aH = aN  # If no splitting, aH is equal to aN
                aN_aH = aN / aH  # Ratio of aN to aH
            
                # Load the machine learning model
                model = joblib.load("model.pkl")
                # Predict radical type using the model
                radical_type_pred = model.predict([[aN, aH, aN_aH]])
                if radical_type_pred == 1:
                    radical_type.append("OH")
                midpoint_B = peaks[0] + (peaks[-1] - peaks[0]) / 2
                g_factor = heights[0] / (9.274 * (10 ** (-24)) * midpoint_B)
                message+=f"G-factor: {g_factor}\nFree radical type: {radical_type}\n"
        if radical_type==[]:
            message+="No specific radicals detected."
        print(radical_type)
        messagebox.showinfo(title='Free Radical', message=message)

           
    def saveCurve(self):
        self.fig.savefig("Plot.png")
          # Open a file dialog for the user to choose the save location and filename
        file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"),("SVG files", "*.svg")])
          # Save the plot to the user's chosen location and filename
        if file_path:
            self.fig.savefig(file_path)
            tkinter.messagebox.showinfo(title='Save', message='File saved successfully' )

        

class App(customtkinter.CTk):
    def __init__(self):
        super().__init__()

        # configure window
        self.title("CustomTkinter complex_example.py")
        self.geometry(f"{1100}x{580}")

        # configure grid layout (4x4)
        self.grid_columnconfigure(0, weight = 2)
        self.grid_columnconfigure(1 , weight = 28)
        self.grid_rowconfigure(0 , weight = 1)
        
        # loading image
        image_path = "test_images"
        self.logo_image = customtkinter.CTkImage(Image.open(os.path.join(image_path, "Tkinter_logo.jpg")), size=(26, 26))
        self.large_test_image = customtkinter.CTkImage(Image.open(os.path.join(image_path, "Tkinter_logo.jpg")), size=(500, 150))
        self.image_icon_image = customtkinter.CTkImage(Image.open(os.path.join(image_path, "Tkinter_logo.jpg")), size=(20, 20))
        self.home_image = customtkinter.CTkImage(light_image=Image.open(os.path.join(image_path, "Tkinter_logo.jpg")),dark_image=Image.open(os.path.join(image_path, "Tkinter_logo.jpg")), size=(20, 20))
        self.chat_image = customtkinter.CTkImage(light_image=Image.open(os.path.join(image_path, "Tkinter_logo.jpg")),dark_image=Image.open(os.path.join(image_path, "Tkinter_logo.jpg")), size=(20, 20))
        self.add_user_image = customtkinter.CTkImage(light_image=Image.open(os.path.join(image_path, "Tkinter_logo.jpg")),dark_image=Image.open(os.path.join(image_path, "Tkinter_logo.jpg")), size=(20, 20))

        # create sidebar frame with widgets
        self.navigation_frame = customtkinter.CTkFrame(self, corner_radius=0)
        self.navigation_frame.grid(row=0,column=0 ,sticky="nsw")
        self.navigation_frame.grid_rowconfigure(4 , weight = 1)

        self.navigation_frame_label = customtkinter.CTkLabel(self.navigation_frame, text="  EPR Analysis", image=self.logo_image,
                                                             compound="left", font=customtkinter.CTkFont(size=15, weight="bold"))
        self.navigation_frame_label.grid(row=0, column=0, padx=20, pady=20)

        self.home_button = customtkinter.CTkButton(self.navigation_frame, corner_radius=0, height=40, border_spacing=10, text="Home",fg_color="transparent", text_color=("gray10", "gray90"), hover_color=("gray70", "gray30"),image=self.home_image, anchor="w", command=self.home_button_event)
        self.home_button.grid(row=1, column=0, sticky="ew")

        self.frame_2_button = customtkinter.CTkButton(self.navigation_frame, corner_radius=0, height=40, border_spacing=10, text="g-factor",fg_color="transparent", text_color=("gray10", "gray90"), hover_color=("gray70", "gray30"),image=self.chat_image, anchor="w", command=self.frame_2_button_event)
        self.frame_2_button.grid(row=2, column=0, sticky="ew")

        #self.frame_3_button = customtkinter.CTkButton(self.navigation_frame, corner_radius=0, height=40, border_spacing=10, text="g-factor",fg_color="transparent", text_color=("gray10", "gray90"), hover_color=("gray70", "gray30"),image=self.add_user_image, anchor="w", command=self.frame_3_button_event)
        #self.frame_3_button.grid(row=3, column=0, sticky="ew")

        self.appearance_mode_menu = customtkinter.CTkOptionMenu(self.navigation_frame, values=["Light", "Dark", "System"],command=self.change_appearance_mode_event)
        self.appearance_mode_menu.grid(row=6, rowspan = 3, column=0, padx=20, pady=20, sticky="s")
        

        self.currFrame = MainFrame(self)
        self.currFrame.grid(row = 0 , column = 1,padx = (10,20) , pady=20,sticky = "nsew")
          

    def open_input_dialog_event(self):
        dialog = customtkinter.CTkInputDialog(text="Type in a number:", title="CTkInputDialog")
        print("CTkInputDialog:", dialog.get_input())

    def change_appearance_mode_event(self, new_appearance_mode: str):
        customtkinter.set_appearance_mode(new_appearance_mode)

    def change_scaling_event(self, new_scaling: str):
        new_scaling_float = int(new_scaling.replace("%", "")) / 100
        customtkinter.set_widget_scaling(new_scaling_float)

    def sidebar_button_event(self):
        print("sidebar_button click")
        
    def home_button_event(self):
            self.currFrame.destroy()
            self.currFrame = MainFrame(self)
            self.currFrame.grid(row=0, column=1, padx=(10, 20), pady=20, sticky="nsew")
    def frame_2_button_event(self):
        self.currFrame.destroy()
        self.currFrame = StatisticalAnalysisFrame(self)  
        self.currFrame.grid(row=0, column=1, padx=(10, 20), pady=20, sticky="nsew")


        
  
    #def frame_3_button_event(self):
        
        #self.currFrame.destroy()
        #self.currFrame = PlotFrame(self)
        #self.currFrame.grid(row = 0 , column = 1,padx = (10,20) , pady=20,sticky = "nsew")
    
    


if __name__ == "__main__":
    app = App()
    app.mainloop()

In [108]:
class StatisticalAnalysisFrame(customtkinter.CTkFrame):
    
    def __init__(self, container):
        
        super().__init__(container)
        
        self.grid_rowconfigure(0 , weight = 5)
        self.grid_rowconfigure(1 , weight = 1)
        self.grid_rowconfigure(2 , weight = 1)
        self.grid_columnconfigure(0 , weight = 1)
        

        # Statistical analysis figure
        
        self.fig = Figure(figsize = (5, 5))
        self.axis = self.fig.add_subplot(111)

        self.canvas = FigureCanvasTkAgg(self.fig,master = self)  
        self.canvas.draw()
        self.canvas.get_tk_widget().grid(row = 0 , column = 0 , padx = (5 , 5) , pady = (5 , 5) , sticky="nsew")
        
        # Buttons
        self.buttonFrame = customtkinter.CTkFrame(self)
        self.buttonFrame.grid(row = 1 , column = 0 , padx = 20 , pady = 2)
        
        self.inputButton = customtkinter.CTkButton(self.buttonFrame , text = "Input data" , command = self.inputData)
        self.inputButton.grid(row = 0 , column = 0 , padx = 10)
        
        self.analyseButton = customtkinter.CTkButton(self.buttonFrame , text = "Analyse" , command = self.analyse)
        self.analyseButton.grid(row = 0 , column = 1 , padx = 10)
        
        self.SaveButton = customtkinter.CTkButton(self.buttonFrame , text = "Save figure" , command = self.saveFigure)
        self.SaveButton.grid(row = 0 , column = 2 , padx = 10)
        

    def inputData(self):
        files = tkinter.filedialog.askopenfilenames(parent=self, title='Choose files', filetypes=[("text files", ".txt"), ("csv files", ".csv")])
        self.field = []
        self.intensity = []
        for ele in files:
            if ele.endswith(".csv"):  # Check if the file is a CSV file
                self.data = pd.read_csv(ele)
                self.plotButton.configure(state="enabled")  # Enable plot feature
            elif ele.endswith(".txt"):  # Check if the file is a text file
                values = np.genfromtxt(ele, names=True, skip_header=2)
                self.data = pd.DataFrame(values)
        # Fill missing values with column mean
        for column in self.data.columns:
            col_mean = self.data[column].mean()
            self.data[column] = self.data[column].fillna(col_mean)
        # Drop the fourth column
        if len(self.data.columns) > 3:
            self.data = self.data.drop(self.data.columns[3], axis=1)
         # Rename columns
        self.data.columns = ['index', 'Field', 'Intensity']
        print(self.data)
    
    def analyse(self):
        series=self.data.isnull().sum()
        for index, value in series.items():
            if value!=0:
                col_mean=self.data[index].mean()
                self.data[index]=self.data[index].fillna(col_mean)
        self.data['g-factor']=689.470305/self.data['Field']  

        self.fig = Figure(figsize=(14, 5))
        
        #self.fig = Figure(figsize = (5, 5))
        #self.axis = self.fig.add_subplot(111)
        
        #self.canvas = FigureCanvasTkAgg(self.fig,master = self)  
        #self.canvas.draw()
        #self.canvas.get_tk_widget().grid(row = 0 , column = 0 , padx = (5 , 5) , pady = (5 , 5) , sticky="nsew")
        #self.axis.clear()
        #self.axis.plot(self.data['Field'] , self.data['g-factor'])
        #self.axis.set_xlabel('Magnetic Field(mT)')
        #self.axis.set_ylabel('g-factor')
        #self.axis.set_title('g-factor v/s Magnetic Field')
        #self.canvas.draw()
        
        self.subplot1 = self.fig.add_subplot(121)  # First subplot
        self.subplot1_visible = True
        self.plot1, = self.subplot1.plot(self.data['Field'], self.data['g-factor'])
        self.subplot1.set_xlabel('g-factor')
        self.subplot1.set_ylabel('Intensity')
        self.subplot1.set_title('g-factor v/s Magnetic Field')

        self.subplot2 = self.fig.add_subplot(122)  # Second subplot
        self.subplot2_visible = True
        self.plot2, = self.subplot2.plot(self.data['Field'], self.data['Intensity'])
        self.subplot2.set_xlabel('Magnetic Field (mT)')
        self.subplot2.set_ylabel('Intensity')
        self.subplot2.set_title('Intensity v/s Magnetic Field')

        self.canvas = FigureCanvasTkAgg(self.fig, master=self)
        self.canvas.draw()
        self.canvas.get_tk_widget().grid(row = 0 , column = 0 , sticky="nsew")
        self.input_button = tkinter.Button(self, text="Input Data", command=self.toggle_plots)
        #self.input_button.pack(padx=10, pady=5)
        self.input_button.grid(row=0, column=0, padx=10, pady=5)
        #print(self.data['g-factor'])
        
    def saveFigure(self):
        
        self.fig.savefig("Plot.png")
          # Open a file dialog for the user to choose the save location and filename
        file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png")])
          # Save the plot to the user's chosen location and filename
        if file_path:
            self.fig.savefig(file_path)
            tkinter.messagebox.showinfo(title='Save', message='File saved successfully' )
    def toggle_plots(self):
        if self.subplot2_visible:
            self.subplot2.clear()
            self.canvas.draw()
            self.subplot2_visible = False
        else:
            self.subplot2.plot(self.data['Field'], self.data['Intensity'])

            # Draw a line at intensity = 0
            zero_intensity_line = [0] * len(self.data['Field'])
            self.subplot2.plot(self.data['Field'], zero_intensity_line, linestyle='--', color='red')

            # Find the crossing point of the peak with the zero intensity line
            crossing_points = self.data[(self.data['Intensity'][:-1] < 0) & (self.data['Intensity'][1:] > 0)]
            if not crossing_points.empty:
                crossing_point = crossing_points.iloc[0]['Field']
                self.subplot2.axvline(x=crossing_point, color='green', linestyle='--')
                # Calculate the final g-factor
                final_g_factor = 0.00714471 / crossing_point
                self.subplot2.text(crossing_point, 0, f'Final g-factor: {final_g_factor:.2f}', color='blue')

        self.canvas.draw()
        self.subplot2_visible = True

        

In [109]:
def toggle_plots(self):
        # Plot the data in the second subplot
        #self.subplot2.plot(self.data['Field'], self.data['Intensity'])
        
        # Plot a line at intensity = 0 in the first subplot (Intensity vs. g-factor)
        zero_intensity_line_g_factor = [0] * len(self.filtered_data['g-factor'])
        self.subplot1.plot(self.filtered_data['g-factor'], zero_intensity_line_g_factor, linestyle='--', color='red', zorder=10)
        crossing_points = []
        intensity_sign = np.sign(self.data['Intensity'])  # Get the sign of intensity
        print(intensity_sign)
        for i in range(1, len(intensity_sign)):
            if intensity_sign[i - 1] != intensity_sign[i]:
                crossing_points.append(i)
        print(crossing_points)
    
        if crossing_points:
            # Take the midpoint of the first crossing point
            crossing_point_index = crossing_points[0]
            crossing_point = self.filtered_data.iloc[crossing_point_index]['g-factor']
        
            # Plot a vertical line at the crossing point
            self.subplot1.axvline(x=crossing_point, color='green', linestyle='--')
        
            # Calculate the final g-factor
            final_g_factor = crossing_point
        
            self.subplot1.text(crossing_point, 0, f'Final g-factor: {final_g_factor:.2f}', color='blue')
            # Display the g-factor value in a message box
            messagebox.showinfo(title='G-Factor', message=f'The final g-factor is: {final_g_factor:.2f}')

            #self.subplot2_visible = True

        # Redraw both subplots and the canvas
        self.subplot1.figure.canvas.draw()
        self.subplot2.figure.canvas.draw()
        self.canvas.draw()

In [ ]:
 def toggle_plots(self):
        # Plot the data in the second subplot
        #self.subplot2.plot(self.data['Field'], self.data['Intensity'])
    
        # Plot a line at intensity = 0 in the first subplot (Intensity vs. g-factor)
        zero_intensity_line_g_factor = [0] * len(self.filtered_data['g-factor'])
        self.subplot1.plot(self.filtered_data['g-factor'], zero_intensity_line_g_factor, linestyle='--', color='red', zorder=10)
    
        # Detect peaks in the intensity data
        peaks, _ = find_peaks(self.filtered_data['Intensity'],height=0.1)
        print(peaks)
        if peaks.size > 0:
            for peak_index in peaks:
                # Interpolate to find the crossing point where intensity is zero
                intensity_values = self.filtered_data['Intensity'].values
                g_factor_values = self.filtered_data['g-factor'].values
                intensity_at_peak = intensity_values[peak_index]
                intensity_before_peak = intensity_values[peak_index - 1]
                g_factor_at_peak = g_factor_values[peak_index]
                g_factor_before_peak = g_factor_values[peak_index - 1]
                slope = (intensity_at_peak - intensity_before_peak) / (g_factor_at_peak - g_factor_before_peak)
                crossing_point_g_factor = g_factor_at_peak - (intensity_at_peak / slope)
            
                # Plot a vertical line at the crossing point
                self.subplot1.axvline(x=crossing_point_g_factor, color='green', linestyle='--')
            
                # Calculate the final g-factor
                final_g_factor = crossing_point_g_factor
                print(final_g_factor)
            
                #self.subplot1.text(crossing_point_g_factor, 0, f'Final g-factor: {final_g_factor:.2f}', color='blue')
                # Display the g-factor value in a message box
                #messagebox.showinfo(title='G-Factor', message=f'The final g-factor is: {final_g_factor:.2f}')

      # Redraw both subplots and the canvas
        self.subplot1.figure.canvas.draw()
        self.subplot2.figure.canvas.draw()
        self.canvas.draw()
    
